In [1]:
import os

import pandas as pd

from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager
from graphrag.query.indexer_adapters import (
    read_indexer_communities,
    read_indexer_entities,
    read_indexer_reports,
)
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch
from graphrag.tokenizer.get_tokenizer import get_tokenizer

In [ ]:
context_builder_params = {
    # 是否在构建上下文时使用社区短摘要（True）或完整报告（False）。
    # 使用完整报告会提供更丰富的上下文信息，但会消耗更多 token 与计算资源；使用摘要能降低成本与上下文大小。
    "use_community_summary": False,
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    # 从数据中用于构建 LLM 上下文的最大 token 数量。根据你所用模型的 token 限额调整该值，
    # 例如当模型限制为 8k token 时，一个常见的设置是 5000；较大模型（32k）可以提高此值以包含更多上下文。
    "max_tokens": 12_000,
    "context_name": "Reports",
}

map_llm_params = {
    # Map 阶段（对每个数据块调用 LLM）的参数。
    # max_tokens：map 阶段每次生成的最大 token，建议设置得比 reduce 阶段小。
    # temperature：控制生成的随机性，0.0 表示确定性输出（用于检索/摘要型任务通常设为 0.0）。
    # response_format：指示 LLM 返回的格式（这里示例为 JSON 对象，方便后续结构化处理）。
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    # Reduce 阶段（整合所有 map 输出，生成最终回答）的参数。
    # reduce 阶段通常需要比 map 更大的 max_tokens 来合并和生成完整回答；请根据模型限制调整。
    "max_tokens": 2000,
    "temperature": 0.0,
}

In [ ]:
api_key = os.environ.get("GRAPHRAG_API_KEY")

# 优先从项目根目录下的 settings.yaml 加载模型配置（这样会使用你在配置里指定的 api_base / 代理地址）
from pathlib import Path
from graphrag.config.load_config import load_config

gr_config = None
try:
    gr_config = load_config(Path.cwd())
    # 全局搜索配置（global_search）通常会引用一个 chat_model_id，默认值为 'default_chat_model'
    model_id = getattr(gr_config.global_search, "chat_model_id", "default_chat_model")
    # get_language_model_config 返回一个已经被验证过的 LanguageModelConfig 实例，该实例包含 api_base、model_provider 等字段（如果 settings.yaml 中已配置）
    config = gr_config.get_language_model_config(model_id)
    print(
        f"Loaded model config from settings.yaml: model_id={model_id}, provider={config.model_provider}, api_base={config.api_base}"
    )
except Exception as e:  # 如果加载 settings.yaml 出现问题则退回到内联配置
    # 加载失败示例：找不到 settings.yaml、配置验证失败、环境变量未设置等。
    print("无法从 settings.yaml 加载模型配置，回退到 notebook 中的内联配置。错误信息：", e)
    # 如果你更喜欢在 notebook 中硬编码代理 / 中转地址，请在下方通过 GRAPHRAG_API_BASE 环境变量或直接修改 api_base 来指定。
    config = LanguageModelConfig(
        api_key=api_key,
        type=ModelType.Chat,
        model_provider="openai",
        model="gpt-4.1",
        api_base=os.environ.get("GRAPHRAG_API_BASE", None),
        max_retries=20,
    )

model = ModelManager().get_or_create_chat_model(
    name="global_search",
    model_type=ModelType.Chat,
    config=config,
)

tokenizer = get_tokenizer(config)

Loaded model config from settings.yaml: model_id=default_chat_model, provider=openai, api_base=https://aihubmix.com/v1


In [ ]:
# 以下变量定义：指定 indexing pipeline 生成的 parquet 文件的位置和表名
INPUT_DIR = "./output/"
COMMUNITY_TABLE = "communities"
COMMUNITY_REPORT_TABLE = "community_reports"
ENTITY_TABLE = "entities"

# COMMUNITY_LEVEL 表示要从 Leiden 社区层级中载入的社区粒度级别。
# 值越大代表更细粒度（更多更小的社区），会使用更多的报告条目，从而提高搜索精度但增加计算成本。
COMMUNITY_LEVEL = 2

In [5]:
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")

communities = read_indexer_communities(community_df, report_df)
reports = read_indexer_reports(report_df, community_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, community_df, COMMUNITY_LEVEL)

print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)

report_df.head()

Total report count: 615
Report count after filtering by community level 2: 519


,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,5674a271f53c42d9bbe8574dc5059d38,613,613,5,604,[],Tang Seng's Pilgrimage and the Quest for Enlig...,The community centers around Tang Seng and his...,# Tang Seng's Pilgrimage and the Quest for Enl...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'Tang Seng is the central cha...,"{\n ""title"": ""Tang Seng's Pilgrimage and th...",2025-11-04,67
1,94ed8074fec642bfb99a08081d9ebab6,614,614,5,604,[],Buddhism and Its Practices,"The community centers around Buddhism, a major...",# Buddhism and Its Practices\n\nThe community ...,7.0,The impact severity rating is high due to Budd...,[{'explanation': 'Buddhism is a major world re...,"{\n ""title"": ""Buddhism and Its Practices"",\...",2025-11-04,2
2,a81ad9895add4000a1e0690c564f4279,603,603,4,527,[],Sha Seng and the Journey to Nanhai and Luo Jia...,The community centers around the character Sha...,# Sha Seng and the Journey to Nanhai and Luo J...,7.5,The impact severity rating is high due to the ...,"[{'explanation': 'Sha Seng, also known as Sand...","{\n ""title"": ""Sha Seng and the Journey to N...",2025-11-04,3
3,b9f1004a3dcf452eab0bb3964118a790,604,604,4,527,"[613, 614]",Journey to the West: Tang Seng and the Quest f...,The community centers around Tang Seng and his...,# Journey to the West: Tang Seng and the Quest...,8.5,The impact severity rating is high due to the ...,[{'explanation': 'Tang Seng is the central cha...,"{\n ""title"": ""Journey to the West: Tang Sen...",2025-11-04,69
4,ff8a58d4721347e2a0d72e59b956525f,605,605,4,552,[],Golden Palace and the Dynamics of Power,The community is centered around the Golden Pa...,# Golden Palace and the Dynamics of Power\n\nT...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'The Golden Palace serves as ...,"{\n ""title"": ""Golden Palace and the Dynamic...",2025-11-04,8


In [ ]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    communities=communities,
    entities=entities,  # 如果不想使用实体计算社区权重，这里可以传入 None；使用实体将会在上下文排序中考虑基于实体的社区权重
    tokenizer=tokenizer,
)

In [ ]:
search_engine = GlobalSearch(
    model=model,
    context_builder=context_builder,
    tokenizer=tokenizer,
    # max_data_tokens：一次构建 LLM 上下文可消费的最大数据 tokens。
    # 根据所使用模型的 token 限制进行调整：例如对 8k 限制的模型，建议设为 5000 左右；对 32k 模型可以更高以包含更多上下文。
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    # allow_general_knowledge：是否允许 LLM 在回答中引入通用知识（模型知识库）以补充报告内容。
    # 开启此项可能有助于回答缺少上下文时的常识性问题，但也有增加幻觉（hallucination）的风险。
    # json_mode：是否启用模型的 JSON 输出模式（如果模型支持，可确保返回结构化输出）。
    # 如果你的模型不支持 JSON 模式，请将其设置为 False。
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    # response_type：期望的回答格式描述（自由文本），可以是 "single paragraph"、"multiple paragraphs"、"prioritized list" 等，便于控制输出风格。
)

In [8]:
result = await search_engine.search("猪八戒爱吃什么？")

print(result.response)

## 猪八戒的饮食偏好

猪八戒，亦称为猪八戒或Zhu Bajie，是《西游记》中的一个重要角色，以其对食物的热爱而闻名。他的饮食偏好不仅反映了他的性格特征，也为故事增添了许多幽默元素。

### 食物种类

猪八戒特别喜欢各种丰盛的食物，尤其是肉类和美味的佳肴。他的饮食习惯中包括：

- **肉类**：猪八戒对肉类的热爱是显而易见的，尤其是猪肉、羊肉等。他在故事中常常被描绘为在丰盛的宴会上大快朵颐，享受各种肉类美食 [Data: Reports (39, 42, 57)]。
- **素食**：除了肉类，猪八戒也会享用素食，如素果品和斋饭，这些食物在故事中象征着团结和共享 [Data: Reports (27, 56)]。
- **特色小吃**：他对一些地方特色小吃，如烧饼等也表现出浓厚的兴趣，这些食物不仅满足了他的食欲，也反映了他对美食的追求 [Data: Reports (51)]。

### 性格特征

猪八戒的贪吃性格是他角色的一个重要方面，常常导致幽默的情节发展。他的贪婪和对食物的渴望使他在旅途中成为了一个喜剧角色，提供了许多轻松的时刻 [Data: Reports (1, 11, 19)]。他的这种性格特征不仅让他在故事中显得更加人性化，也使得他与其他角色之间的互动更加生动有趣。

### 文化意义

在《西游记》中，食物不仅仅是生存的需要，更是文化和人际关系的象征。猪八戒的饮食习惯反映了中国文化中对食物的重视，尤其是在团体活动和社交场合中，食物的分享和享用是建立联系的重要方式 [Data: Reports (34, 32)]。

### 结论

综上所述，猪八戒的饮食偏好主要集中在肉类和丰盛的美食上，同时也包括一些素食选项。他的贪吃性格为故事增添了幽默感，并反映了食物在文化和人际关系中的重要性。这些特征使得猪八戒成为了一个深受读者喜爱的角色，展现了人性中的贪婪与欢乐。
